<a href="https://colab.research.google.com/github/sipocz/pid_time_series/blob/main/PID_NN_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [111]:
!pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [112]:
import pandas as pd
import os
import seaborn as sns
import wandb
import numpy as np

In [113]:
from google.colab import drive

In [114]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [115]:
#user = "Anna"
user = "SL"

In [116]:
if user=="Anna":
    path = "/content/drive/MyDrive/Egyetem_MSc/Diplomamunka/2022Anna/Datapipeline/SINT3_415/3D_transposed/"
else:
    path = "/content/drive/MyDrive/2022Anna/Datapipeline/SINT3_415/3D_transposed/"

In [117]:
#user = "Anna"
user = "SL"
uzem = "Szint3"
data_source="SINT3_415/3D_transposed"
#fname="72C03_TC_error_toNN.csv"
fname_good = "415_SC_3D_part"
fname_bad = "415_SC_3D_part"
fname_good_ext=["1.csv","3.csv","5.csv","7.csv"]
fname_bad_ext=["2.csv","4.csv","6.csv","8.csv"]

In [118]:
# Elérési út a 415_SC_error-hoz
if user=="Anna":
    path_good = "/content/drive/MyDrive/Egyetem_MSc/Diplomamunka/2022Anna/Datapipeline/" + data_source + "/" + fname_good
    path_bad = "/content/drive/MyDrive/Egyetem_MSc/Diplomamunka/2022Anna/Datapipeline/" + data_source + "/" + fname_bad
    path_fig = "/content/drive/MyDrive/Egyetem_MSc/Diplomamunka/2022Anna/Datapipeline/plots/"
else:
    path_good = "/content/drive/MyDrive/2022Anna/Datapipeline/" + data_source + "/" + fname_good 
    path_bad = "/content/drive/MyDrive/2022Anna/Datapipeline/" + data_source + "/" + fname_bad
    path_fig = "/content/drive/MyDrive/2022Anna/Datapipeline/plots/"


In [119]:
print(path_good)
print(path_bad)

/content/drive/MyDrive/2022Anna/Datapipeline/SINT3_415/3D_transposed/415_SC_3D_part
/content/drive/MyDrive/2022Anna/Datapipeline/SINT3_415/3D_transposed/415_SC_3D_part


In [120]:
cols=[str(i) for i in range(60)]

In [121]:
#Listát készít a driveon lévő mappa exceleinek elérési útvonaláról (debug módban kiirja az elérési utakat)
def create_file_list(path_name, extension=".csv",debug=False):
 
  dir_path = os.path.dirname(path_name)
  eleresi_utak = []

  for root, dirs, files in os.walk(dir_path):
    for file in files:
      if file.endswith(extension):
        if debug:
          print(root+'/'+str(file))
        eleresi_utak.append(root+'/'+str(file))
  return (eleresi_utak)

In [122]:
#File lista meghívás
file_list=create_file_list(path, ".csv")

In [123]:
df_good=pd.DataFrame(columns=cols)
df_bad=pd.DataFrame(columns=cols)
print(df_good.head())
for i,goods in enumerate(fname_good_ext):
    df_good_tmp = pd.read_csv(path_good+goods,usecols=None )
    df_good_tmp.columns=cols
    df_good=pd.concat([df_good,df_good_tmp],axis=0,)
    df_bad_tmp = pd.read_csv(path_bad+fname_bad_ext[i],usecols=None)
    df_bad_tmp.columns=cols
    df_bad=pd.concat([df_bad,df_bad_tmp],axis=0)

Empty DataFrame
Columns: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59]
Index: []

[0 rows x 60 columns]


In [124]:
len(df_good)

2646

In [125]:
print(df_good.isnull().values.any())
print(df_bad.isnull().values.any())

False
False


In [126]:

import matplotlib.pyplot as plot

df_ = df_good

# You must normalize the data before applying the fit method
df_good_normalized=(df_ - df_.mean()) / df_.std()

# Normalize bad data with the good data parameters
df_bad_normalized=(df_bad - df_.mean()) / df_.std()

In [127]:
df_.std()

0     26.253038
1     26.245471
2     26.237871
3     26.230238
4     26.222572
5     26.214873
6     26.207142
7     26.199377
8     26.191580
9     26.183749
10    26.175886
11    26.167989
12    26.160060
13    26.152097
14    26.144101
15    26.136072
16    26.128009
17    26.119913
18    26.111784
19    26.103622
20    26.027930
21    26.028361
22    26.031677
23    26.032146
24    26.024675
25    26.017621
26    26.010087
27    26.002321
28    25.994570
29    25.986785
30    25.978969
31    25.971119
32    25.963237
33    25.955321
34    25.947374
35    25.939393
36    25.931379
37    25.923332
38    25.915253
39    25.907889
40     3.694510
41     3.498658
42     3.404934
43     3.377770
44     3.376818
45     3.376750
46     3.376750
47     3.376750
48     3.376750
49     3.376750
50     3.376750
51     3.376751
52     3.376751
53     3.376751
54     3.376751
55     3.376751
56     3.376752
57     3.376752
58     3.376753
59     3.376794
dtype: float64

In [128]:
df_good_normalized["state"]=0
df_bad_normalized["state"]=1

In [129]:
df_all_normalized=pd.concat([df_good_normalized,df_bad_normalized],axis=0)

In [130]:
# Ez jó, kiinduló csv sora 60 széles, egymás után Sp Out Error
X_good = np.ndarray(shape=(len(df_good),20,3))
X_good[:,:,0] = df_good_normalized.iloc[:, 0:20]
X_good[:,:,1] = df_good_normalized.iloc[:, 20:40]
X_good[:,:,2] = df_good_normalized.iloc[:, 40:60]


In [131]:
y_all=df_all_normalized["state"]


In [132]:
X_all = np.ndarray(shape=(len(df_all_normalized),20,3))
X_all[:,:,0] = df_all_normalized.iloc[:, 0:20]
X_all[:,:,1] = df_all_normalized.iloc[:, 20:40]
X_all[:,:,2] = df_all_normalized.iloc[:, 40:60]

In [133]:
_N1_=1100 #70  #700
_N2_=118 #12  #120
_lr_=0.001
_batch_size_=32
_drop1_=0.5
_drop2_=0.7
_epochs_=9500

In [134]:
import wandb

In [135]:
config = {"lr": _lr_, "batch_size": _batch_size_,"architecture": "LSTM", 
          "depth": 2,
          "layer1":_N1_,  "layer2":_N2_, 
          "drop1":_drop1_,"drop2":_drop2_,
          "epochs":_epochs_
          
          
          }

In [136]:
wandb.init(project="pid_3d_lstm", entity="sipoczlaszlo",config=config)

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [142]:
from wandb.keras import WandbMetricsLogger
fname="./model_PID_NN_LSTM_"
callbacks = [#callback_LR,
        WandbMetricsLogger(),
        ModelCheckpoint(filepath=fname+"_{epoch}"+"_{loss:.4f}_{val_loss:.4f}_.hdf5", monitor='val_loss',
                        verbose=2, save_best_only=True, mode='min')]

In [138]:
from keras.engine.base_layer import regularizers
from keras.layers import InputLayer, Dense, LSTM, Input, Dropout
from keras.models import Sequential, Model
from keras.optimizers import SGD,Adam,Adamax,Nadam,Ftrl,Adadelta
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint
from keras.backend import clear_session
from tensorflow.keras.losses import mean_absolute_percentage_error, huber,kld
from sklearn.model_selection import train_test_split
from keras.callbacks import LearningRateScheduler

clear_session()

kernel_reg_1=tf.keras.regularizers.L2(0.1)

input_size=20


input1=Input(shape=(input_size,3))
l1_out=LSTM(input_size*2,)(input1) 
l2_out=Dropout(_drop1_)(l1_out)


l3_out=Dense(_N2_,activation="swish",kernel_initializer='glorot_uniform',)(l2_out) #kernel_initializer='lecun_normal',
l4_out=Dropout(_drop2_)(l3_out)

pred=Dense(1, activation="sigmoid",)(l4_out)

model = Model(inputs=input1, outputs=pred)
optimizer=Adamax(learning_rate=_lr_,) #

model.compile(loss='binary_crossentropy',
    optimizer=optimizer,
        metrics=["accuracy"])

In [139]:
X_train,X_test,y_train,y_test=train_test_split(X_all,y_all,train_size=0.9,shuffle=True,random_state=33)

In [ ]:
history = model.fit(
    x=X_train,
    y=y_train,
    validation_data=(X_test,y_test),
    epochs=1500,
    batch_size=10,
    callbacks=callbacks
   )

Epoch 1/1500
509/510 [============================>.] - ETA: 0s - loss: 0.2629 - accuracy: 0.8864
Epoch 1: val_loss improved from inf to 0.23139, saving model to ./model_PID_NN_LSTM__1_0.2628_0.2314_.hdf5
510/510 [==============================] - 6s 12ms/step - loss: 0.2628 - accuracy: 0.8865 - val_loss: 0.2314 - val_accuracy: 0.8905
Epoch 2/1500
509/510 [============================>.] - ETA: 0s - loss: 0.2604 - accuracy: 0.8886
Epoch 2: val_loss improved from 0.23139 to 0.22770, saving model to ./model_PID_NN_LSTM__2_0.2604_0.2277_.hdf5
510/510 [==============================] - 7s 15ms/step - loss: 0.2604 - accuracy: 0.8886 - val_loss: 0.2277 - val_accuracy: 0.8975
Epoch 3/1500
508/510 [============================>.] - ETA: 0s - loss: 0.2602 - accuracy: 0.8852
Epoch 3: val_loss improved from 0.22770 to 0.22687, saving model to ./model_PID_NN_LSTM__3_0.2600_0.2269_.hdf5
510/510 [==============================] - 6s 11ms/step - loss: 0.2600 - accuracy: 0.8853 - val_loss: 0.2269 - va